In [ ]:
# |default_exp wrappers

In [ ]:
# |export
import logging
from functools import cached_property
from time import time

import gym
import numpy as np
import torch
import torch.nn.functional as F
from fastcore.basics import class2attr
from scipy.spatial.transform import Rotation
from tmenv.gbx import ghost_wr_from_map_uid
from tmenv.trajectory import Trajectory
from torchvision.transforms import Resize

# Wrappers
> Env wrappers for tmenv

In [ ]:
# |export
class StateWrapper(gym.Wrapper):
    def __init__(self, env, state_space):
        super().__init__(env)
        self.observation_space[self.observation_name] = state_space

    @property
    def observation_name(self):
        return class2attr(self, "(StateWrapper|Wrapper|State)")

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        observation[self.observation_name] = self.state(observation)
        return observation, reward, done, info

    def respawn(self, launched=True):
        observation = self.env.respawn(launched)
        observation[self.observation_name] = self.state(observation)
        return observation

    def reset(self, **kwargs):
        observation = self.env.reset(**kwargs)
        observation[self.observation_name] = self.state(observation)
        return observation

    def state(self, observation):
        raise NotImplementedError()

In [ ]:
# |export
class VisualState(StateWrapper):
    def __init__(self, env, shape, div=255.0):
        if isinstance(shape, int):
            shape = (shape, shape)
        self.shape, self.div = shape, div
        self.resize = Resize(shape, antialias=True)
        super().__init__(env, gym.spaces.Box(0, 1, (3, *shape)))

    def state(self, observation):
        return (
            self.resize(self.game.capture_tensor().to(self.device, non_blocking=True))
            .float()
            .div(self.div)
        )

In [ ]:
# |export
def vec2np(vec):
    return np.array(vec, dtype=np.float32)

In [ ]:
# |export
XYZ = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])


def vec2rotation(forward, left, up):
    r, d = Rotation.align_vectors(XYZ, [forward, left, up])
    assert d < 1e-2, f"rmsd is {d} wich is greater that 1e-2"
    return r


def angle_diff(a, b):
    return np.arctan2(np.sin(b - a), np.cos(b - a))


class RotationEnvWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.rotation = None

    def _rotation(self, env_obs):
        previous_rotation = vec2rotation(
            env_obs["previous"]["forward"],
            env_obs["previous"]["left"],
            env_obs["previous"]["up"],
        )
        previous_yaw, previous_pitch, previous_roll = previous_rotation.as_euler("yxz")
        self.rotation = vec2rotation(
            env_obs["forward"],
            env_obs["left"],
            env_obs["up"],
        )
        yaw, pitch, roll = self.rotation.as_euler("yxz")
        time_diff = env_obs["_updatetime"] - env_obs["previous"]["_updatetime"]
        if time_diff < 1e-4:
            yaw_speed = 0
            pitch_speed = 0
            roll_speed = 0
        else:
            yaw_speed = angle_diff(previous_yaw, yaw) / time_diff
            pitch_speed = angle_diff(previous_pitch, pitch) / time_diff
            roll_speed = angle_diff(previous_roll, roll) / time_diff
        velocity = self.rotation.apply(env_obs["velocity"])
        velocity_norm = np.linalg.norm(velocity)
        velocity_normed = velocity / velocity_norm if velocity_norm > 1e-3 else velocity

        return dict(
            yaw=yaw.item(),
            pitch=pitch.item(),
            roll=roll.item(),
            yaw_speed=yaw_speed.item(),
            pitch_speed=pitch_speed.item(),
            roll_speed=roll_speed.item(),
            velocity=velocity.tolist(),
            velocity_norm=velocity_norm.item(),
            velocity_normed=velocity_normed.tolist(),
        )

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        env = observation["env"]
        env["rotation"] = self._rotation(env)
        return observation, reward, done, info

    def respawn(self, launched=True):
        observation = self.env.respawn(launched)
        env = observation["env"]
        env["rotation"] = self._rotation(env)
        return observation

    def reset(self, **kwargs):
        observation = self.env.reset(**kwargs)
        env = observation["env"]
        env["rotation"] = self._rotation(env)
        return observation

In [ ]:
# |export
class TrajectoryEnvWrapper(gym.Wrapper):
    def __init__(
        self,
        env,
        max_distance,
        ghost_types=["author", "wr"],
        jump_z_coef=0.5,  # Distance to trajectory on z axis coef when the reference trajectory is jumping
        minimum_speed=5,
        maximum_travel=300,
    ):
        super().__init__(env)
        self.max_distance = max_distance
        self.ghost_types = ghost_types
        self.jump_z_coef = jump_z_coef
        self.minimum_speed = minimum_speed
        self.maximum_travel = maximum_travel
        self.ghost_data = None
        self.trajectory = None

    def load_map(self, map_data):
        self.env.load_map(map_data)
        for ghost_type in self.ghost_types:
            if ghost_type in self.map_data["ghosts"]:
                self.ghost_data = self.map_data["ghosts"][ghost_type]
                logging.info(
                    f"Using {ghost_type} ghost for map {self.map_data['name']}"
                )
                break
        assert (
            self.ghost_data is not None
        ), f"No ghost found for map {self.map_data['name']}"
        self.trajectory = Trajectory(self.ghost_data)

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        env = observation["env"]
        traj_obs = self.trajectory.observation(
            vec2np(env["position"]), len(self.checkpoints), self.standing_cp_offset, self.jump_z_coef
        )
        traj_obs["progress"] = (
            1.0 if env["finished"] else min(0.999, traj_obs["progress"])
        )

        traj_obs["too_far"] = self.max_distance < traj_obs["weighted_distance"]
        traj_obs["furthest"] = self.progress < traj_obs["progress"]
        # prevent ultra low speed to count as progress (turtle or facing wall)
        traj_obs["furthest"] &= self.minimum_speed < (env["velocity_norm"] * 3.6)
        # prevent big cut to count as progress (these are not true cut, AI is stuck afterwards or take wrong cp)
        traj_obs["furthest"] &= traj_obs["travel"] - self.travel < self.maximum_travel
        traj_obs["mistake"] = traj_obs["too_far"] or not traj_obs["furthest"]

        if traj_obs["mistake"]:
            traj_obs["mistake_duration"] = time() - self.mistake_start_time
        else:
            self.mistake_start_time = time()
            traj_obs["mistake_duration"] = 0

        if traj_obs["furthest"]:
            traj_obs["step_travel"] = traj_obs["travel"] - self.travel
            self.travel = traj_obs["travel"]
            self.progress = traj_obs["progress"]
            if self.max_progress < self.progress:
                self.max_progress = self.progress
        else:
            traj_obs["step_travel"] = 0
            traj_obs["progress"] = self.progress

        env["trajectory"] = traj_obs
        info["progress"] = traj_obs["progress"]
        info["max_progress"] = self.max_progress
        return observation, reward, done, info

    def respawn(self, launched=True):
        observation = self.env.respawn(launched)
        env = observation["env"]
        traj_obs = self.trajectory.observation(
            vec2np(env["position"]), len(self.checkpoints), self.standing_cp_offset, self.jump_z_coef
        )

        self.progress = traj_obs["progress"]
        self.mistake_start_time = time()
        self.travel = traj_obs["travel"]
        traj_obs["missed_checkpoint"] = False
        traj_obs["too_far"] = False
        traj_obs["furthest"] = True
        traj_obs["step_travel"] = 0
        traj_obs["mistake_duration"] = 0

        env["trajectory"] = traj_obs
        return observation

    def reset(self, **kwargs):
        observation = self.env.reset(**kwargs)
        env = observation["env"]
        traj_obs = self.trajectory.observation(
            vec2np(env["position"]), len(self.checkpoints), self.standing_cp_offset, self.jump_z_coef
        )

        self.progress = 0
        self.max_progress = 0
        self.mistake_start_time = time()
        self.travel = 0
        traj_obs["missed_checkpoint"] = False
        traj_obs["too_far"] = False
        traj_obs["furthest"] = True
        traj_obs["step_travel"] = 0
        traj_obs["mistake_duration"] = 0
        traj_obs["progress"] = 0

        env["trajectory"] = traj_obs
        return observation

In [ ]:
# |export
class TelemetryState(StateWrapper):
    def __init__(self, env, telemetry_lag_divisor=20, env_lag_divisor=0.1):
        self.shape = (66,)
        super().__init__(env, gym.spaces.Box(-1.0, 1.0, self.shape))
        self.telemetry_lag_divisor = telemetry_lag_divisor
        self.env_lag_divisor = env_lag_divisor
        assert hasattr(self, "rotation"), "Env must have RotationEnvWrapper"
        assert hasattr(self, "trajectory"), "Env must have TrajectoryEnvWrapper"

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        observation[self.observation_name] = self.state(
            observation, self.action_handler.action_dict(action)
        )
        return observation, reward, done, info

    def state(self, observation, action=dict(accelerate=False, brake=False, steer=0.0)):
        env = observation["env"]
        return torch.tensor(
            [
                action["accelerate"],
                action["brake"],
                action["steer"],
                env["telemetry_lag"] / self.telemetry_lag_divisor,
                (time() - env["observation_start_time"]) / self.env_lag_divisor,
                env["finished"],
                env["accel_coef"],
                env["control_coef"],
                env["gravity_coef"],
                env["adherence_coef"],
                env["upwardness"],
                env["wheels_contact_count"] / 4.0,
                env["wheels_skidding_count"] / 4.0,
                env["flying_duration"] > 0,
                env["skidding_duration"] > 0,
                env["handicap_no_gas_duration"] > 0,
                env["handicap_force_gas_duration"] > 0,
                env["handicap_no_brakes_duration"] > 0,
                env["handicap_no_steering_duration"] > 0,
                env["handicap_no_grip_duration"] > 0,
                *[env["reactor_boost_level"] == i for i in range(3)],
                *[env["reactor_boost_type"] == i for i in range(4)],
                env["reactor_inputs_x"],
                env["is_ground_contact"],
                env["is_reactor_ground_mode"],
                env["is_wheels_burning"],
                env["ground_dist"] / 10,
                *[env["gear"] == i for i in range(1, 6)],
                env["engine_on"],
                env["is_turbo"],
                env["turbo_time"],
                env["bullet_time_normed"],
                env["simulation_time_coef"],
                env["air_brake_normed"],
                env["spoiler_open_normed"],
                env["wings_open_normed"],
                env["is_top_contact"],
                env["wetness"],
                env["water_immersion_coef"],
                env["water_over_dist_normed"],
                env["front_speed"] * 3.6 / 500,
                env["front_speed"] < 0,
                env["velocity_norm"] * 3.6 / 500,
                env["rpm"] / 11000.0,
                env["side_speed"] * 3.6 / 500,
                env["rotation"]["pitch"],
                env["rotation"]["roll"],
                env["rotation"]["yaw_speed"],
                env["rotation"]["pitch_speed"],
                env["rotation"]["roll_speed"],
                env["rotation"]["velocity_norm"],
                *env["rotation"]["velocity_normed"],
                env["trajectory"]["missed_checkpoint"],
                env["trajectory"]["furthest"],
                env["trajectory"]["too_far"],
            ],
            dtype=torch.float,
        ).to(self.device, non_blocking=True)

In [ ]:
# |export
class WheelState(StateWrapper):
    num_contact_classes = 82
    wheel_types = ["fl_wheel", "fr_wheel", "rl_wheel", "rr_wheel"]

    def __init__(self, env):
        self.shape = (self.num_contact_classes + 8, 4)
        super().__init__(env, gym.spaces.Box(-1.0, 1.0, self.shape))

    def wheel_state(self, observation, wheel_type):
        env = observation["env"]
        wheel_state = torch.tensor(
            [
                env[wheel_type + "_steer_angle"],
                env[wheel_type + "_rotation"] / 10000,
                env[wheel_type + "_damper_len"],
                env[wheel_type + "_slip_coef"],
                env[wheel_type + "_dirt"],
                env[wheel_type + "_break_coef"],
                env[wheel_type + "_tire_wear"],
                env[wheel_type + "_icing"],
            ],
            dtype=torch.float,
        ).to(self.device, non_blocking=True)
        contact_state = F.one_hot(
            torch.tensor(env[wheel_type + "_contact_material"]).to(
                self.device, non_blocking=True
            ),
            num_classes=self.num_contact_classes,
        ).squeeze()
        return torch.cat([wheel_state, contact_state])

    def state(self, observation):
        states = [self.wheel_state(observation, t) for t in self.wheel_types]
        return torch.stack(
            states,
            dim=1,
        )

In [ ]:
# |export
class TrajectoryState(StateWrapper):
    def __init__(self, env, nb_positions, spacing):
        self.shape = (4, nb_positions)
        super().__init__(env, gym.spaces.Box(-1.0, 1.0, self.shape))
        self.nb_positions = nb_positions
        self.spacing = spacing
        assert hasattr(self, "trajectory"), "Env must have TrajectoryEnvWrapper"
        assert hasattr(self, "rotation"), "Env must have RotationEnvWrapper"

    def state(self, observation):
        return self.trajectory.frame_trajectory(
            vec2np(observation["env"]["position"]),
            self.rotation,
            len(self.checkpoints),
            self.standing_cp_offset,
            self.nb_positions,
            self.spacing,
            device=self.device,
        )

class AlignedState(StateWrapper):
    def __init__(self, env):
        super().__init__(env, gym.spaces.Box(-1.0, 1.0, (1,)))
        assert hasattr(self, "trajectory"), "Env must have TrajectoryEnvWrapper"

    def state(self, observation):
        aligned = self.trajectory.aligned(
            vec2np(observation["env"]["position"]),
            len(self.checkpoints),
            self.standing_cp_offset,
            vec2np(observation["env"]["velocity"]),
            device=self.device,
        )
        observation["env"]["aligned"] = bool(aligned)
        return aligned

In [ ]:
# |export
class PositionalEncoder:
    def __init__(self, d_model, min_freq=1e-4, device="cpu"):
        self.d_model = d_model
        self.min_freq = min_freq
        self.device = device
        self.freqs = min_freq ** (
            2 * torch.arange(d_model).div(2, rounding_mode="floor") / d_model
        ).reshape(1, -1).to(self.device)

    def __call__(self, position):
        pos_enc = position.reshape(-1, 1) * self.freqs
        pos_enc[:, ::2] = torch.cos(pos_enc[:, ::2])
        pos_enc[:, 1::2] = torch.sin(pos_enc[:, 1::2])
        return pos_enc

class MapState(StateWrapper):
    def __init__(self, env, d_model=16):
        self.shape = (3 * d_model,)
        super().__init__(env, gym.spaces.Box(-1.0, 1.0, self.shape))
        self.penc = PositionalEncoder(d_model, 1e-5, self.device)

    def state(self, observation):
        pos = torch.tensor(
            observation["env"]["position"], dtype=torch.float, device=self.device
        )
        pos = self.penc(pos).flatten()
        return pos

In [ ]:
# |export
class TrajectoryDoneWrapper(gym.Wrapper):
    def __init__(
        self,
        env,
        max_mistake_duration,
        respawn_sequence=["launched", "standing"] * 2,
        give_up_before_first_cp=True,
    ):
        super().__init__(env)
        self.max_mistake_duration = max_mistake_duration
        self.respawn_sequence = respawn_sequence
        self.give_up_before_first_cp = give_up_before_first_cp
        self.max_respawn = len(respawn_sequence)
        assert hasattr(self, "trajectory"), "Env must have TrajectoryEnvWrapper"

    def update_observation(self, observation):
        observation["env"]["trajectory"][
            "max_mistake_duration"
        ] = self.max_mistake_duration
        if (not self.give_up_before_first_cp or self.checkpoints) and not observation[
            "env"
        ]["finished"]:
            observation["env"]["trajectory"]["max_respawn"] = self.max_respawn
        else:
            observation["env"]["trajectory"]["max_respawn"] = 0
        observation["env"]["trajectory"]["respawn_count"] = self.respawn_count
        observation["env"]["trajectory"]["respawn_progress"] = self.respawn_progress

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        traj_obs = observation["env"]["trajectory"]
        exceed_mistake_duration = (
            self.max_mistake_duration < traj_obs["mistake_duration"]
        )
        if exceed_mistake_duration and not done:
            if (
                not self.give_up_before_first_cp or self.checkpoints
            ) and self.respawn_count < self.max_respawn:
                info["must_respawn"] = True
            else:
                done = True

        if observation["env"]["crossed_checkpoint"] or observation["env"]["finished"]:
            self.respawn_count = 0
            self.respawn_progress = observation["env"]["trajectory"]["progress"]

        self.update_observation(observation)
        info["total_respawn_count"] = self.total_respawn_count
        info["respawn_type"] = self.respawn_sequence[
            self.respawn_count % self.max_respawn
        ]
        return observation, reward, done, info

    def respawn(self, launched=True):
        observation = self.env.respawn(launched)
        self.respawn_count += 1
        self.total_respawn_count += 1
        self.respawn_progress = observation["env"]["trajectory"]["progress"]
        self.update_observation(observation)
        return observation

    def reset(self, **kwargs):
        observation = self.env.reset(**kwargs)
        self.respawn_count = 0
        self.total_respawn_count = 0
        self.respawn_progress = 0
        self.update_observation(observation)
        return observation

In [ ]:
# |export
class TrajectoryTravelRewardWrapper(gym.Wrapper):
    def __init__(
        self,
        env,
        mistake_reward,
        discount_factor,
        penalty_threshold=0.0,
    ):
        super().__init__(env)
        self.mistake_reward = mistake_reward
        self.discount_factor = discount_factor
        self.penalty_threshold = penalty_threshold
        self.div = 100/3.6 # Normalize the reward, the AI will spend a lot of time around 100kmh
        self.min_reward = self.mistake_reward / (1 -  self.discount_factor) / self.div
        self.max_reward = (1000/3.6) / (1 -  self.discount_factor) / self.div
        assert hasattr(self, "trajectory"), "Env must have TrajectoryEnvWrapper"

    def distance_penalty(self, distance):
        if distance < self.penalty_threshold:
            return 1.0
        elif self.max_distance < distance:
            return 0.0
        return 1.0 - (distance - self.penalty_threshold) / (
            self.max_distance - self.penalty_threshold
        )

    def travel_reward(self, env_obs):
        reward = env_obs["trajectory"]["step_travel"]
        reward *= self.distance_penalty(env_obs["trajectory"]["weighted_distance"])
        reward /= env_obs["_updatetime"] - self.previous_updatetime
        return reward

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        if observation["env"]["finished"]:
            reward += observation["env"]["velocity_norm"] / (1 -  self.discount_factor)
        elif done or info["must_respawn"]:
            reward += self.mistake_reward / (1 -  self.discount_factor)
        elif observation["env"]["trajectory"]["mistake"]:
            reward += self.mistake_reward
        else:
            reward += self.travel_reward(observation["env"])

        reward /= self.div

        self.previous_updatetime = observation["env"]["_updatetime"]
        return observation, reward, done, info

    def respawn(self, launched=True):
        observation = self.env.respawn(launched)
        self.previous_updatetime = observation["env"]["_updatetime"]
        return observation

    def reset(self, **kwargs):
        observation = self.env.reset(**kwargs)
        self.previous_updatetime = observation["env"]["_updatetime"]
        return observation

In [ ]:
# |export
class EpisodeInfoWrapper(gym.Wrapper):
    def reset(self, **kwargs):
        observation = self.env.reset(**kwargs)
        self.segments = []
        self.segment_reward = 0
        self.segment_step_count = 0
        self.total_reward = 0
        self.total_step_count = 0
        self.segment_racetime = 0
        self.no_respawn_racetime_offset = 0
        self.no_respawn_reward_offset = 0
        self.no_respawn_step_count_offset = 0
        self.medal = "unfinished"
        return observation

    def add_segment(self, type, racetime, duration):
        self.segments.append(
            dict(
                type=type,
                racetime=racetime,
                duration=duration,
                reward=self.segment_reward,
                step_count=self.segment_step_count,
            )
        )
        self.segment_reward = 0
        self.segment_step_count = 0
        self.segment_racetime = racetime

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        self.segment_reward += reward
        self.segment_step_count += 1
        self.total_reward += reward
        self.total_step_count += 1
        if observation["env"]["crossed_checkpoint"]:
            duration = info["racetime"] - self.segment_racetime
            self.add_segment("checkpoint", info["racetime"], duration)
        if observation["env"]["finished"]:
            info["finish_racetime"] = info["racetime"]
            info["no_respawn_finish_racetime"] = (
                info["racetime"] - self.no_respawn_racetime_offset
            )
            if info["racetime"] <= self.map_data["medals"]["author"]:
                self.medal = "author"
            elif info["racetime"] <= self.map_data["medals"]["gold"]:
                self.medal = "gold"
            elif info["racetime"] <= self.map_data["medals"]["silver"]:
                self.medal = "silver"
            elif info["racetime"] <= self.map_data["medals"]["bronze"]:
                self.medal = "bronze"
            else:
                self.medal = "no_medal"
        info |= dict(
            segments=self.segments,
            total_reward=self.total_reward,
            total_step_count=self.total_step_count,
            no_respawn_racetime=info["racetime"] - self.no_respawn_racetime_offset,
            no_respawn_reward=self.total_reward - self.no_respawn_reward_offset,
            no_respawn_step_count=self.total_step_count
            - self.no_respawn_step_count_offset,
            medal=self.medal,
        )
        return observation, reward, done, info

    def respawn(self, launched=True):
        observation = self.env.respawn(launched)
        duration = observation["env"]["racetime"] - self.segment_racetime
        self.no_respawn_racetime_offset += duration
        self.no_respawn_reward_offset += self.segment_reward
        self.no_respawn_step_count_offset += self.segment_step_count
        self.add_segment("respawn", observation["env"]["racetime"], duration)
        return observation

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()